# Get State and Update State

Once you start [checkpointing](./persistence.ipynb) your graphs, you can easily
**get** or **update** the state of the agent at any point in time. This permits
a few things:

1. You can surface a state during an interrupt to a user to let them accept an
   action.
2. You can **rewind** the graph to reproduce or avoid issues.
3. You can **modify** the state to embed your agent into a larger system, or to
   let the user better control its actions.

The key methods used for this functionality are:

- [getState](https://langchain-ai.github.io/langgraphjs/reference/classes/pregel.Pregel.html#getState):
  fetch the values from the target config
- [updateState](https://langchain-ai.github.io/langgraphjs/reference/classes/pregel.Pregel.html#updateState):
  apply the given values to the target state

**Note:** this requires passing in a checkpointer.

<!-- Example:
```javascript
TODO
...
``` -->

This works for
[StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html)
and all its subclasses, such as
[MessageGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MessageGraph.html).

Below is an example.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        In this how-to, we will create our agent from scratch to be transparent (but verbose). You can accomplish similar functionality using the <code>createReactAgent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) constructor. This may be more appropriate if you are used to LangChain’s <a href="https://js.langchain.com/v0.2/docs/langgraph/#agentexecutor">AgentExecutor</a> class.
    </p>
</div>

## Setup

This guide will use OpenAI's GPT-4o model. We will optionally set our API key
for [LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

In [1]:
// Deno.env.set("OPENAI_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Time Travel: LangGraphJS");


## Define the state

The state is the interface for all of the nodes in our graph.


In [2]:
import { BaseMessage } from "@langchain/core/messages";

interface IState {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
}

// This defines the agent state
const graphState: IState = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};


## Set up the tools

We will first define the tools we want to use. For this simple example, we will
use create a placeholder search engine. However, it is really easy to create
your own tools - see documentation
[here](https://js.langchain.com/v0.2/docs/how_to/custom_tools) on how to do
that.


In [3]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder for the actual implementation
    return "Cold, with a low of 13 ℃";
  },
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];


"Cold, with a low of 13 ℃"

We can now wrap these tools in a simple
[ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html).
This object will actually run the tools (functions) whenever they are invoked by
our LLM.


In [4]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);


## Set up the model

Now we will load the
[chat model](https://js.langchain.com/v0.2/docs/concepts/#chat-models).

1. It should work with messages. We will represent all agent state in the form
   of messages, so it needs to be able to work well with them.
2. It should work with
   [tool calling](https://js.langchain.com/v0.2/docs/how_to/tool_calling/#passing-tools-to-llms),
   meaning it can return function arguments in its response.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        These model requirements are not general requirements for using LangGraph - they are just requirements for this one example.
    </p>
</div>

In [23]:
import { ChatOpenAI } from "@langchain/openai";

// const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" });
const model = new ChatOpenAI({ model: "gpt-4o" });


After we've done this, we should make sure the model knows that it has these
tools available to call. We can do this by calling
[bindTools](https://v01.api.js.langchain.com/classes/langchain_core_language_models_chat_models.BaseChatModel.html#bindTools).


In [24]:
const boundModel = model.bindTools(tools);
boundModel.kwargs;


{
  tools: [
    {
      type: "function",
      function: {
        name: "search",
        description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
        parameters: {
          type: "object",
          properties: [Object],
          required: [Array],
          additionalProperties: false,
          "$schema": "http://json-schema.org/draft-07/schema#"
        }
      }
    }
  ]
}

## Define the graph

We can now put it all together. Time travel requires a checkpointer to save the
state - otherwise you wouldn't have anything go `get` or `update`. We will use
the
[MemorySaver](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MemorySaver.html),
which "saves" checkpoints in-memory.

In [25]:
import { AIMessage, BaseMessage } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";
import { END, MemorySaver, START, StateGraph } from "@langchain/langgraph";

const routeMessage = (state: { messages: Array<BaseMessage> }) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // If no tools are called, we can finish (respond to the user)
  if (!lastMessage.tool_calls || lastMessage.tool_calls.length === 0) {
    return END;
  }
  // Otherwise if there is, we continue and call the tools
  return "tools";
};

const callModel = async (
  state: { messages: Array<BaseMessage> },
  config: RunnableConfig,
) => {
  const { messages } = state;
  const response = await boundModel.invoke(messages, config);
  return { messages: [response] };
};

const workflow = new StateGraph({
  channels: graphState,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("tools", toolNode);

// Set the entrypoint as `agent`
workflow.addEdge(START, "agent");
workflow.addConditionalEdges("agent", routeMessage);
workflow.addEdge("tools", "agent");

// Here we only save in-memory
let memory = new MemorySaver();
const graph = workflow.compile({ checkpointer: memory });

StateGraph {
  nodes: {
    agent: RunnableLambda {
      lc_serializable: false,
      lc_kwargs: { func: [AsyncFunction: callModel] },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      func: [AsyncFunction: callModel]
    },
    tools: ToolNode {
      lc_serializable: false,
      lc_kwargs: {},
      lc_runnable: true,
      name: "tools",
      lc_namespace: [ "langgraph" ],
      func: [Function: func],
      tags: undefined,
      config: { tags: [] },
      trace: true,
      recurse: true,
      tools: [
        DynamicStructuredTool {
          lc_serializable: false,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: "search",
          verbose: false,
          callbacks: undefined,
          tags: [],
          metadata: {},
          returnDirect: false,
          description: "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
          fu

## Interacting with the Agent

We can now interact with the agent. Between interactions you can get and update
state.

In [26]:
let config = { configurable: { thread_id: "conversation-num-1" } };
let inputs = { messages: [["user", "Hi I'm Jo."]] };
for await (
  const { messages } of await graph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

[ "user", "Hi I'm Jo." ]
-----



Hello Jo! How can I assist you today?
-----



See LangSmith example run here
https://smith.langchain.com/public/b3feb09b-bcd2-4ad5-ad1d-414106148448/r

Here you can see the "agent" node ran, and then our edge returned `__end__` so
the graph stopped execution there.

Let's check the current graph state.

In [27]:
let checkpoint = await graph.getState(config);
checkpoint.values;


{
  messages: [
    [ "user", "Hi I'm Jo." ],
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Hello Jo! How can I assist you today?",
        tool_calls: [],
        invalid_tool_calls: [],
        additional_kwargs: {},
        response_metadata: { tokenUsage: [Object], finish_reason: "stop" }
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Hello Jo! How can I assist you today?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {
        tokenUsage: { completionTokens: 11, promptTokens: 68, totalTokens: 79 },
        finish_reason: "stop"
      },
      tool_calls: [],
      invalid_tool_calls: []
    }
  ]
}

The current state is the two messages we've seen above, 1. the HumanMessage we
sent in, 2. the AIMessage we got back from the model.

The `next` values are empty since the graph has terminated (transitioned to the
`__end__`).

In [28]:
checkpoint.next;


[]

## Let's get it to execute a tool

When we call the graph again, it will create a checkpoint after each internal
execution step. Let's get it to run a tool, then look at the checkpoint.

In [29]:
inputs = { messages: [["user", "What's the weather like in SF currently?"]] };
for await (
  const { messages } of await graph.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

[ "user", "What's the weather like in SF currently?" ]
-----

[
  {
    name: "search",
    args: { query: "current weather in San Francisco" },
    id: "call_9nR1UWZgVwW3BtEeMSmjTkHp"
  }
]
-----

Cold, with a low of 13 ℃
-----

The current weather in San Francisco is cool, with a temperature of around 13°C (55°F). Do you need any other information?
-----



See the trace of the above execution here:
https://smith.langchain.com/public/0ef426fd-0da1-4c02-a50b-64ae1e68338e/r We can
see it planned the tool execution (ie the "agent" node), then "should_continue"
edge returned "continue" so we proceeded to "action" node, which executed the
tool, and then "agent" node emitted the final response, which made
"should_continue" edge return "end". Let's see how we can have more control over
this.

### Pause before tools

If you notice below, we now will add `interruptBefore=["action"]` - this means
that before any actions are taken we pause. This is a great moment to allow the
user to correct and update the state! This is very useful when you want to have
a human-in-the-loop to validate (and potentially change) the action to take.

In [30]:
memory = new MemorySaver();
const graphWithInterrupt = workflow.compile({
  checkpointer: memory,
  interruptBefore: ["tools"],
});

inputs = { messages: [["user", "What's the weather like in SF currently?"]] };
for await (
  const { messages } of await graphWithInterrupt.stream(inputs, {
    ...config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

[ "user", "What's the weather like in SF currently?" ]
-----

[
  {
    name: "search",
    args: { query: "current weather in San Francisco" },
    id: "call_ONueXiVpSg1lC5MIFDVmDswq"
  }
]
-----



## Get State

You can fetch the latest graph checkpoint using
[`getState(config)`](https://langchain-ai.github.io/langgraphjs/reference/classes/pregel.Pregel.html#getState).

In [31]:
let snapshot = await graphWithInterrupt.getState(config);
snapshot.next;


[ "tools" ]

## Resume

You can resume by running the graph with a `null` input. The checkpoint is
loaded, and with no new inputs, it will execute as if no interrupt had occurred.

In [33]:
for await (
  const { messages } of await graphWithInterrupt.stream(null, {
    ...snapshot.config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Cold, with a low of 13 ℃
-----

The current weather in San Francisco is cold, with a temperature of around 13℃.
-----



## Check full history

Let's browse the history of this thread, from newest to oldest.


In [34]:
let toReplay = null;
for await (const state of graphWithInterrupt.getStateHistory(config)) {
  console.log(state);
  console.log("--");
  if (state.values?.messages?.length === 2) {
    toReplay = state;
  }
}


{
  values: {
    messages: [
      [ "user", "What's the weather like in SF currently?" ],
      AIMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "",
          tool_calls: [Array],
          invalid_tool_calls: [],
          additional_kwargs: [Object],
          response_metadata: [Object]
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "",
        name: undefined,
        additional_kwargs: { tool_calls: [Array] },
        response_metadata: { tokenUsage: [Object], finish_reason: "tool_calls" },
        tool_calls: [ [Object] ],
        invalid_tool_calls: []
      },
      ToolMessage {
        lc_serializable: true,
        lc_kwargs: {
          name: "search",
          content: "Cold, with a low of 13 ℃",
          tool_call_id: "call_ONueXiVpSg1lC5MIFDVmDswq",
          additional_kwargs: {},
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "

## Replay a past state

To replay from this place we just need to pass its config back to the agent.


In [35]:
for await (
  const { messages } of await graphWithInterrupt.stream(null, {
    ...toReplay.config,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

Cold, with a low of 13 ℃
-----

The current weather in San Francisco is cold, with a low of 13 ℃.
-----



## Branch off a past state

Using LangGraph's checkpointing, you can do more than just replay past states.
You can branch off previous locations to let the agent explore alternate
trajectories or to let a user "version control" changes in a workflow.

#### First, update a previous checkpoint

Updating the state will create a **new** snapshot by applying the update to the
previous checkpoint. Let's **add a tool message** to simulate calling the tool.

In [66]:
import { ToolMessage } from "@langchain/core/messages";

const tool_calls =
  toReplay.values.messages[toReplay.values.messages.length - 1].tool_calls;
const branchConfig = await graphWithInterrupt.updateState(
  toReplay.config,
  {
    messages: [
      new ToolMessage("It's sunny out, with a high of 38 ℃.", tool_calls[0].id),
    ],
  },
  // Updates are applied "as if" they were coming from a node. By default,
  // the updates will come from the last node to run. In our case, we want to treat
  // this update as if it came from the tools node, so that the next node to run will be
  // the agent.
  "tools",
);

const branchState = await graphWithInterrupt.getState(branchConfig);
console.log(branchState.values);
console.log(branchState.next);

{
  messages: [
    [ "user", "What's the weather like in SF currently?" ],
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "",
        tool_calls: [ [Object] ],
        invalid_tool_calls: [],
        additional_kwargs: { tool_calls: [Array] },
        response_metadata: { tokenUsage: [Object], finish_reason: "tool_calls" }
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "",
      name: undefined,
      additional_kwargs: { tool_calls: [ [Object] ] },
      response_metadata: {
        tokenUsage: { completionTokens: 17, promptTokens: 72, totalTokens: 89 },
        finish_reason: "tool_calls"
      },
      tool_calls: [
        {
          name: "search",
          args: [Object],
          id: "call_ONueXiVpSg1lC5MIFDVmDswq"
        }
      ],
      invalid_tool_calls: []
    },
    ToolMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "It's sunny out, with a high of 38 ℃.",
        tool_call_id:

#### Now you can run from this branch

Just use the updated config (containing the new checkpoint ID). The trajectory
will follow the new branch.

In [67]:
for await (
  const { messages } of await graphWithInterrupt.stream(null, {
    ...branchConfig,
    streamMode: "values",
  })
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  } else if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  } else {
    console.log(msg);
  }
  console.log("-----\n");
}

The current weather in San Francisco is sunny with a high of 38°C (100°F).
-----

